In [1]:
import random
import copy

class Team:
  def __init__(self, name, wins, games, points):
    self._name = name
    self._points = points
    self._wins = wins
    self._games = games
    if(wins != 0 and games != 0):
        self._win_rate = wins/games
    else:
        self._win_rate = 0
        
  def update_win_rate(self):
    self._win_rate = self._wins/self._games    
    
  def win(self):   
    self._games = self._games + 1
    self._wins = self._wins + 1
    self.update_win_rate()  
    
  def lose(self):
    self._games = self._games + 1 
    self.update_win_rate() 
    
  def tie(self):
    self._games = self._games + 1 
    self.update_win_rate()    
    
  def win_points(self):
    self._points = self._points + 3
    self._games = self._games + 1
    self._wins = self._wins + 1
    self.update_win_rate()  
    
  def lose_points(self):
    self._games = self._games + 1 
    self.update_win_rate() 
    
  def tie_points(self):
    self._points = self._points + 1
    self._games = self._games + 1 
    self.update_win_rate()    
    
  def __repr__(self):
    return (self._name +" : " + str(self._points) + " / "+ str(self._wins)+ " / "+ str(self._games)+ " / "+ str(self._win_rate) + "\n")


def search_team_one(gameString,teamList,team_one):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team._name == teams[0]):      
            return team

def search_team_two(gameString,teamList,team_two):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team._name == teams[1]):      
            return team

In [11]:
def simulate_game(team_one,team_two,current_team_list):
  status = "UNDEFINED"
  rng = random.uniform(0,1)
  rng_two = random.uniform(0,1)

  if(team_one._win_rate >= rng):
    if(team_two._win_rate < rng_two):
      team_one.win_points()
      team_two.lose_points()
      status = "ONE WON"   
    else:
      team_one.tie_points()
      team_two.tie_points()
      status = "TIE"
  else:
    if(team_two._win_rate >= rng_two):
      team_two.win_points()
      team_one.lose_points()
      status = "TWO WON"   
    else:
      team_one.tie_points()
      team_two.tie_points()
      status = "TIE"

def simulate_game_elm(team_one,team_two):
  status = "UNDEFINED"
  while(status == "UNDEFINED" or status == "TIE"):
        rng = random.uniform(0,1)
        rng_two = random.uniform(0,1)  
        
        if(team_one._win_rate >= rng):
            if(team_two._win_rate < rng_two):      
              team_one.win()  
              status = "WIN"
              return team_one
            else:     
              status = "TIE"
        else:
            if(team_two._win_rate >= rng_two):      
              team_two.win()              
              status = "WIN"
              return team_two
            else:      
              status = "TIE" 
            
def initialize_list(original_team_list):
    base_name = "group"
    original_team_list = [[],[],[],[],[],[],[],[]]
    for j in range (0,8):
        current_file = base_name + str(j)+ ".txt"
        f = open(current_file, "r",encoding="utf-8-sig")        
        for line in f.readlines():
            ele = line.split(";")
            if(ele[0] != '\n'):
                ele[3] = ele[3][:-1]         
                temp_team = Team(ele[0],int(ele[2]),int(ele[3]),int(ele[1]))
                original_team_list[j].append(temp_team)
    return original_team_list

def ordenate_list_groups(current_team_list):
    for group in current_team_list:
        group.sort(key=lambda x: x._points, reverse=True)  
    return current_team_list

def extract_winners_groups(current_team_list):
    winner_list = []
    for group in current_team_list:
        winner_list.append(group[0])
        winner_list.append(group[1])
    return winner_list

In [29]:
%%time

k = 1000
current_team_list = [[],[],[],[],[],[],[],[]]
times_brazil_s_p = 0 
second_phase_teams = []
second_phase_frames = []
brazil_group_frames = []
coup_winners = []
            
for i in range(0,k):   
  current_team_list = initialize_list(current_team_list)
  t = open("jogos_copa.txt", "r",encoding="utf-8-sig")
  team_one = Team("",0,0,0)
  team_two = Team("",0,0,0)          

  for game in t.readlines():  
    team_one = search_team_one(game,current_team_list,team_one)
    team_two = search_team_two(game,current_team_list,team_two)  
    simulate_game(team_one,team_two,current_team_list)
    
  current_team_list = ordenate_list_groups(current_team_list)
  second_phase_teams = extract_winners_groups(current_team_list)
    
  if(current_team_list[6][0]._name == "Brasil" or current_team_list[6][1]._name == "Brasil"):
    times_brazil_s_p = times_brazil_s_p + 1
    
  found = False
  current_group_string = ""
  for team in current_team_list[6]:
    current_group_string = current_group_string + team._name + ":" + str(team._points) + ";"
  for groupAndCounter in brazil_group_frames:     
    if(groupAndCounter[0] == current_group_string):
      found = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found):
    brazil_group_frames.append([current_group_string,1]) 
    
  found_win_list =  False
  current_win_list = ""
  for team in second_phase_teams:
    current_win_list = current_win_list + team._name + ";"
  for groupAndCounter in second_phase_frames:     
    if(groupAndCounter[0] == current_win_list):
      found_win_list = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found_win_list):
    second_phase_frames.append([current_win_list,1])
    
  first_a_team = second_phase_teams[0]
  second_a_team = second_phase_teams[1]
  first_b_team = second_phase_teams[2]
  second_b_team = second_phase_teams[3]
  first_c_team = second_phase_teams[4]
  second_c_team = second_phase_teams[5]
  first_d_team = second_phase_teams[6]
  second_d_team = second_phase_teams[7]    
  first_e_team = second_phase_teams[8]
  second_e_team = second_phase_teams[9]
  first_f_team = second_phase_teams[10]
  second_f_team = second_phase_teams[11]
  first_g_team = second_phase_teams[12]
  second_g_team = second_phase_teams[13]
  first_h_team = second_phase_teams[14]
  second_h_team = second_phase_teams[15]   
     
  team_49 = simulate_game_elm(first_a_team,second_b_team)     
  team_50 = simulate_game_elm(first_c_team,second_d_team) 
  team_51 = simulate_game_elm(first_d_team,second_c_team)   
  team_52 = simulate_game_elm(first_b_team,second_a_team)    
  team_53 = simulate_game_elm(first_e_team,second_f_team)  
  team_54 = simulate_game_elm(first_g_team,second_h_team) 
  team_55 = simulate_game_elm(first_f_team,second_e_team)   
  team_56 = simulate_game_elm(first_h_team,second_g_team)   
    
  team_58 = simulate_game_elm(team_53,team_54)  
  team_57 = simulate_game_elm(team_49,team_50) 
  team_60 = simulate_game_elm(team_55,team_56)   
  team_59 = simulate_game_elm(team_51,team_52) 
    
  team_61 = simulate_game_elm(team_57,team_58)   
  team_62 = simulate_game_elm(team_59,team_60)
    
  team_63 = simulate_game_elm(team_61,team_62) 
  
  found_winner = False
  for teamAndCounter in coup_winners:     
     if(teamAndCounter[0] == team_63._name):
         found_winner = True
         teamAndCounter[1] = teamAndCounter[1] + 1
  if(not found_winner):
     coup_winners.append([team_63._name,1])
 

coup_winners.sort(key=lambda x: x[1], reverse=True)   
for group in coup_winners:
    group[1] = (group[1] / k) * 100

brazil_group_frames.sort(key=lambda x: x[1], reverse=True)   
for group in brazil_group_frames:
    group[1] = (group[1] / k) * 100

second_phase_frames.sort(key=lambda x: x[1], reverse=True)   
for group in second_phase_frames:
    group[1] = (group[1] / k) * 100    

CPU times: user 848 ms, sys: 40 ms, total: 888 ms
Wall time: 888 ms


In [30]:
print("->Porcentagem de vezes que o Brasil classificou para 2 fase: "+ str(times_brazil_s_p/k *100)+"%.")
print("->Quadros e porcentagens do grupo do Brasil:")
for frame in brazil_group_frames[:3]:
    print(frame)
print("->Quadros e porcentagens da segunda fase:")
for frame in second_phase_frames[:3]:
    print(frame)    
print("->Campeões:")
for frame in coup_winners[:10]:
    print(frame)  
  

->Porcentagem de vezes que o Brasil classificou para 2 fase: 86.2%.
->Quadros e porcentagens do grupo do Brazil:
['Brasil:7;Sérvia:3;Suíça:2;Camarões:2;', 2.1]
['Brasil:5;Sérvia:3;Suíça:3;Camarões:2;', 2.0]
['Brasil:9;Sérvia:2;Suíça:2;Camarões:2;', 2.0]
->Quadros e porcentagens da segunda fase:
['Equador;Senegal;Estados Unidos;Gales;Arábia Saudita;Polônia;França;Tunísia;Costa Rica;Alemanha;Croácia;Marrocos;Brasil;Sérvia;Coreia do Sul;Uruguai;', 0.1]
['Catar;Equador;Inglaterra;Irã;Arábia Saudita;Polônia;Dinamarca;Tunísia;Japão;Costa Rica;Bélgica;Canadá;Brasil;Sérvia;Portugal;Gana;', 0.1]
['Catar;Senegal;Irã;Inglaterra;Polônia;Argentina;França;Dinamarca;Espanha;Japão;Marrocos;Bélgica;Brasil;Sérvia;Gana;Uruguai;', 0.1]
->Campeões:
['Brasil', 22.0]
['Inglaterra', 9.0]
['Espanha', 7.5]
['Alemanha', 6.9]
['Argentina', 4.3999999999999995]
['México', 4.3999999999999995]
['Irã', 4.0]
['Austrália', 3.4000000000000004]
['França', 3.4000000000000004]
['Croácia', 3.2]
